# Zalando scraper

In [132]:
## GOOD TO KNOW 

# website = https://zalando.nl/pre-owned-mode-heren/index.html
# article class = _0mW-4D _0xLoFW JT3_zV mo6ZnF _78xIQ- U4aOaA
# figure class = heWLCX
# tag product brand names = <span> 
# inspecting a product gives us: <a class="_LM JT3_zV CKDt_l CKDt_l LyRfpJ"href="https://www.zalando.nl/g-star-overhemd-blue-zir01hbly-001.html" rel="">


In [1]:
# Import packages
from bs4 import BeautifulSoup
import requests
from time import sleep
from datetime import datetime
import csv
import pandas as pd

In [2]:
import requests
from bs4 import BeautifulSoup

#debug = {'verbose': sys.stderr}
user_agent = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36 Edg/98.0.1108.62'}  

In [3]:
# Make selenium and chromedriver work

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

#driver = webdriver.Chrome()
driver = webdriver.Chrome(ChromeDriverManager().install())




====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/sannevanettinger/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


In [71]:
import time

driver.get("https://www.zalando.nl/pre-owned-mode-heren/")
time.sleep(10)
print("done")

done


In [72]:
from bs4 import BeautifulSoup
res = driver.page_source.encode('utf-8')
soup = BeautifulSoup(res, "html.parser")

In [73]:
# accept button on the website (close pop-ups before proceeding)
driver.find_elements_by_class_name("uc-btn-primary")

[<selenium.webdriver.remote.webelement.WebElement (session="7e2615a9dd4d6f1d5b52ce29e7f77a42", element="8b989cb9-0907-465f-98ee-60048861744d")>]

In [89]:
products = soup.find_all("div", class_= "DT5BTM w8MdNG cYylcv QylWsg _75qWlu iOzucJ JT3_zV DvypSJ")

print(len(products))
sleep(1)

84


In [93]:
## this one is wrong - according to Hannes - since the page_urls and product_urls cannot be determined, selenium formula not used

from bs4 import BeautifulSoup
res = driver.page_source.encode('utf-8')
soup = BeautifulSoup(res, "html.parser")

# Create a function to collect the product_urls
def create_product_urls(page_urls):
    product_urls = []
    cnt=1
    for page_url in page_urls:
        cnt+=1
        driver.get(page_url)
        time.sleep(5)
        res = driver.page_source.encode('utf-8')
        soup = BeautifulSoup(res, "html.parser")
        #f=open('website_'+str(cnt)+'.html', 'w', encoding='utf-8')
        #f.write(res.text)
        #f.close()
        products = soup.find_all("div", class_= "DT5BTM w8MdNG cYylcv QylWsg _75qWlu iOzucJ JT3_zV DvypSJ")       
        print(len(products))
        sleep(1)

        for product in products:
            try:
                product_url = product.find("a").attrs["href"]
                print(product_url)
                product_urls.append(product_url)
            except:
                print("dit is geen product")
                
    return product_urls

In [92]:
## not right yet, goal is to scroll down the pages 

from time import sleep
products = []

for _ in range(5):
    res = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(res, "html.parser")
    
    # total number of products in current view
    num_products_view = len(soup.find_all(attrs={"href": "product"}))
        
    # add products to list
    for counter in range(num_products_view):
        products.append(soup.find_all(attrs={"href": "product"})[counter].find_all(attrs={"href": "auto"})[4].text)
    
    # scroll down the page
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    
    # pause for 5 seconds
    sleep(1)

print(num_products_view)

0


In [38]:
## this works right

products = soup.find_all("div", class_= "DT5BTM w8MdNG cYylcv QylWsg _75qWlu iOzucJ JT3_zV DvypSJ")       
product_urls=[]
for product in products:
    try:
        product_url = product.find("a").attrs["href"]
        print(product_url)
        product_urls.append(product_url)
    except:
        print("dit is geen product")
len(product_urls)

https://www.zalando.nl/moods-of-norway-shorts-grey-zir01t473-001.html
https://www.zalando.nl/columbia-shorts-blue-zir01ryti-001.html
https://www.zalando.nl/abercrombie-and-fitch-shorts-black-zir01t62m-001.html
https://www.zalando.nl/columbia-shorts-dark-grey-zir01t3xb-001.html
https://www.zalando.nl/jack-and-jones-sweater-green-zir01t5s0-001.html
https://www.zalando.nl/jacks-sportswear-sweater-grey-zir01t3v7-001.html
https://www.zalando.nl/joop-trui-purple-zir01t4t6-001.html
https://www.zalando.nl/gant-rugger-trui-blue-zir01t4o2-001.html
https://www.zalando.nl/tom-tailor-sweater-met-rits-blue-zir01t54h-001.html
https://www.zalando.nl/qs-by-soliver-trui-blue-zir01t6l5-001.html
https://www.zalando.nl/key-largo-trui-grey-zir01t65p-001.html
https://www.zalando.nl/marc-opolo-poloshirt-grey-zir01t6ti-001.html
https://www.zalando.nl/banana-republic-trui-blue-zir01t4py-001.html
https://www.zalando.nl/samsoee-samsoee-shorts-red-zir01rxhq-001.html
https://www.zalando.nl/next-overhemd-blue-zir01r

84

In [83]:
# is this true? 

scroll_counter = 0 
last_height = 0
driver.get("https://www.zalando.nl/pre-owned-mode-heren/")

# running this cell may take a minute or two
while True:    
    current_height = driver.execute_script('return document.body.scrollHeight')
    print("current height: " + str(current_height))

    if current_height == last_height:
        break

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    scroll_counter += 1
    last_height = current_height
    print("last height: " + str(last_height))
    sleep(1)
    
print(f"The number scrolls required to scrape all products is: {scroll_counter}")

current height: 18483
last height: 18483
current height: 18592
last height: 18592
current height: 18592
The number scrolls required to scrape all products is: 2


In [20]:
# Create a function to collect all page urls
base_url = "https://www.zalando.nl/pre-owned-mode-heren/?p="
counter = 1

# Now create the function
def generate_page_urls(base_url, num_pages):
    page_urls = []
    
    for counter in range (1, num_pages + 1):
        full_url = base_url + str(counter) + "&order=activation_date"
        page_urls.append(full_url)
    
    return page_urls

In [29]:
page_urls = generate_page_urls(base_url, 2) 

generate_page_urls("https://www.zalando.nl/pre-owned-mode-heren/?p=", 20)

['https://www.zalando.nl/pre-owned-mode-heren/?p=1&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=2&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=3&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=4&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=5&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=6&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=7&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=8&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=9&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=10&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=11&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=12&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=13&order=activation_date',
 'https://www.zalando

In [84]:
import requests
from bs4 import BeautifulSoup

# Create a function to collect the product_urls
def create_product_urls(page_urls):
    product_urls = []
    cnt=1
    for page_url in page_urls:
        cnt+=1
        res = requests.get(page_url, headers = user_agent)
        soup = BeautifulSoup(res.text, "html.parser")
        f=open('website_'+str(cnt)+'.html', 'w', encoding='utf-8')
        f.write(res.text)
        f.close()
        products = soup.find_all("div", class_= "DT5BTM w8MdNG cYylcv QylWsg _75qWlu iOzucJ JT3_zV DvypSJ")       
        print(len(products))
        sleep(1)

        for product in products:
            try:
                product_url = product.find("a").attrs["href"]
                #print(product_url)
                product_urls.append(product_url)
            except:
                print("dit is geen product")
                    
    return product_urls

In [85]:
import requests
from bs4 import BeautifulSoup

# scraping information from product pages

def get_product_pages(products):
    product_pages = []
    
    for url in product_urls: 
        res = requests.get(url, headers = user_agent)
        soup = BeautifulSoup(res.text, "html.parser")
        attributes = soup.find_all(class_="DT5BTM VHXqc_ rceRmQ _4NtqZU mIlIve")
        delivery = soup.find(class_="PBR5VH _0xLoFW EJ4MLB")
        
        for attribute in attributes:    
            product = attribute.find("h3", class_="mt1kvu ka2E9k uMhVZi FxZV-M SZKKsK pVrzNP _5Yd-hZ").text
            category = attribute.find("span", class_="EKabf7 R_QwOV").text
            price = attribute.find("span", class_="uqkIZw ka2E9k uMhVZi FxZV-M RYghuO pVrzNP")
            size = attribute.find("span", class_="_7Cm1F9 ka2E9k uMhVZi dgII7d RYghuO pVrzNP").text
            delivery_time = delivery.find("span", class_="u-6V88 ka2E9k uMhVZi dgII7d RYghuO pVrzNP").text
            delivery_time = delivery_time.replace("werkdagen", "")
            delivery_time = delivery_time.replace(" ", "")
            product_pages.append({"url": url,
                                 "brand": product,
                                 "category": category,
                                 "price": price,
                                 "size": size,
                                 "delivery time": delivery_time})
   
    return product_pages
print("done")

done


In [86]:
product_attributes = get_product_pages(product_urls)
product_attributes

[{'url': 'https://www.zalando.nl/moods-of-norway-shorts-grey-zir01t473-001.html',
  'brand': 'Moods of Norway',
  'category': 'Shorts',
  'price': <span class="uqkIZw ka2E9k uMhVZi FxZV-M RYghuO pVrzNP">€ 25,50</span>,
  'size': '50',
  'delivery time': '3-6'},
 {'url': 'https://www.zalando.nl/abercrombie-and-fitch-shorts-black-zir01t62m-001.html',
  'brand': 'Abercrombie & Fitch',
  'category': 'Shorts',
  'price': <span class="uqkIZw ka2E9k uMhVZi FxZV-M RYghuO pVrzNP">€ 12,50</span>,
  'size': '33',
  'delivery time': '3-6'},
 {'url': 'https://www.zalando.nl/qs-by-soliver-trui-blue-zir01t6l5-001.html',
  'brand': 'QS by s.Oliver',
  'category': 'Trui',
  'price': <span class="uqkIZw ka2E9k uMhVZi FxZV-M RYghuO pVrzNP">€ 13,60</span>,
  'size': 'M',
  'delivery time': '3-6'},
 {'url': 'https://www.zalando.nl/gaastra-overhemd-blue-zir01rvok-001.html',
  'brand': 'Gaastra',
  'category': 'Overhemd',
  'price': <span class="uqkIZw ka2E9k uMhVZi FxZV-M RYghuO pVrzNP">€ 21,80</span>,
  's

In [87]:
from datetime import datetime

now = datetime.now()
print(now)

2022-03-15 19:42:40.492786


In [88]:
import csv 

with open("product_description_pre_owned.csv", "w", encoding = 'utf-8', newline='') as csv_file: 
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["url", "brand", "category", "price", "size", "delivery time", "now"])
    now = datetime.now()
    for product in product_attributes: 
        writer.writerow([product['url'], product['brand'], product['category'], product['price'], product['size'], product['delivery time'], now])
print('done!')

done!
